In [1]:
import logging
import datetime
from logging.handlers import TimedRotatingFileHandler
from logging.handlers import RotatingFileHandler
from sklearn.externals import joblib
from collections import OrderedDict
import sys
from dayu.hooks.oss_hook import OSSHook

from dayu.hooks.hive_server_hook import HiveServerHook
from dayu.hooks.hive_cli_hook import HiveCliHook

import pandas as pd 
import numpy as np
from fastFM import sgd
from scipy import sparse
from scipy.sparse import csc_matrix
import time
from sklearn.metrics import auc,accuracy_score,roc_curve,roc_auc_score
from sklearn.metrics import confusion_matrix,recall_score
from sklearn import  metrics
from sklearn.linear_model import LogisticRegression

def split_table_name(datain):
    new_cols = []
    for column in datain.columns:
        if(len(column.split('.'))<2):
            return datain
        tb_name, col_name = column.split('.')
        new_cols.append((column, col_name))
    datain = datain.rename(columns=dict(new_cols))
    return datain

def read_from_hive2(output_file_name,insql,dtype):
    filename = output_file_name
    filepath = curr_dir+filename
    hive = HiveServerHook("warehouse_hive")
    hive.to_csv(insql,filepath , delimiter=',',lineterminator='\n', output_header=True)
    outdata = pd.read_csv(filepath, header=0,dtype=dtype)
    # 去除列名中带有的表名
    outdata = split_table_name(outdata)
    return outdata

curr_date = str(datetime.datetime.now())[0:10]
logging.info('当前日期: %s'%(curr_date))
curr_dir = '/home/souche/qiongjiu/hgc/'
curr_date=str(datetime.datetime.now())[0:10]
hive_cli = HiveCliHook("warehouse_hive")

DAYU_HOME : /home/souche/projects/datacenter-etl-v2


/home/souche/projects/calculation/py3dev_new/lib/python3.5/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


[2020-11-04 10:52:44,989] {driver:120} INFO - Generating grammar tables from /usr/lib/python3.5/lib2to3/Grammar.txt
[2020-11-04 10:52:45,016] {driver:120} INFO - Generating grammar tables from /usr/lib/python3.5/lib2to3/PatternGrammar.txt
[2020-11-04 10:52:46,000] {<ipython-input-1-c2e46d8a12df>:45} INFO - 当前日期: 2020-11-04


In [2]:
sql_1 = '''
SELECT model_code,
       age AS MONTH,
       pay_time,
       residual AS canzhi
FROM db_alg.mid_car_dfc_sale_order_flag 
WHERE model_code IS NOT NULL
  AND city_code IS NOT NULL
  AND trade_type !='B2B'
  AND flag =0
'''
dtype={'model_code':str}
dfc_sales= read_from_hive2('mid_car_dfc_sale_order_flag',sql_1,dtype)
#dfc_sales = load_data_from_hive(sql_1)  # 零售数据

sql_2 = '''
SELECT model_code,
       age AS MONTH,
       pay_time,
       residual AS canzhi
FROM db_alg.mid_car_dfc_purchase_order_flag
WHERE model_code IS NOT NULL
  AND city_code IS NOT NULL
  AND flag =0
'''
dtype={'model_code':str}
dfc_purchase= read_from_hive2('mid_car_dfc_purchase_order_flag',sql_2,dtype)
#dfc_purchase = load_data_from_hive(sql_2)  # 采购数据

car_data_columns= ["country_id","manufacturer","battery_quality_mile","battery_quality_time","brand_code","brand_name","car_body",
"cylinder_number","driving_mode","engine_volume_l","fuel_form","gear_box_type","guide_price","height","import_type",
"intake_type","length","level","max_power","max_torque","model_code","model_name","quality_mile","quality_time","rate",
"rate_count","seat_number_top","series_code","series_level","series_name","wheel_base","width","year","down_market_time",
"production_time","shutdown_time","time_to_market","year_1","year_2","year_3","year_4","year_5","year_6","year_7","year_8",
"year_9","year_10","year_11","year_12","year_13","year_14","year_15","year_16"]
car_model = pd.read_csv(curr_dir+'all_car_model_sep_word_to_sql',header=None,sep='\t',names=car_data_columns)

#car_model = load_data_from_hive(sql_1)  # 车型库


[2020-11-04 10:52:48,773] {hiveserver2:138} INFO - Using database default as default
[2020-11-04 10:52:48,916] {hive_server_hook:112} INFO - Running query: 
SELECT model_code,
       age AS MONTH,
       pay_time,
       residual AS canzhi
FROM db_alg.mid_car_dfc_sale_order_flag 
WHERE model_code IS NOT NULL
  AND city_code IS NOT NULL
  AND trade_type !='B2B'
  AND flag =0

[2020-11-04 10:53:13,342] {hive_server_hook:162} INFO - Written 10000 rows so far.
[2020-11-04 10:53:13,723] {hive_server_hook:162} INFO - Written 20000 rows so far.
[2020-11-04 10:53:14,115] {hive_server_hook:162} INFO - Written 30000 rows so far.
[2020-11-04 10:53:14,495] {hive_server_hook:162} INFO - Written 40000 rows so far.
[2020-11-04 10:53:14,853] {hive_server_hook:162} INFO - Written 50000 rows so far.
[2020-11-04 10:53:15,236] {hive_server_hook:162} INFO - Written 60000 rows so far.
[2020-11-04 10:53:15,625] {hive_server_hook:162} INFO - Written 70000 rows so far.
[2020-11-04 10:53:16,022] {hive_server_ho

[2020-11-04 10:53:49,090] {hive_server_hook:162} INFO - Written 950000 rows so far.
[2020-11-04 10:53:49,482] {hive_server_hook:162} INFO - Written 960000 rows so far.
[2020-11-04 10:53:49,864] {hive_server_hook:162} INFO - Written 970000 rows so far.
[2020-11-04 10:53:50,246] {hive_server_hook:162} INFO - Written 980000 rows so far.
[2020-11-04 10:53:50,601] {hive_server_hook:162} INFO - Written 990000 rows so far.
[2020-11-04 10:53:50,989] {hive_server_hook:162} INFO - Written 1000000 rows so far.
[2020-11-04 10:53:51,383] {hive_server_hook:162} INFO - Written 1010000 rows so far.
[2020-11-04 10:53:51,791] {hive_server_hook:162} INFO - Written 1020000 rows so far.
[2020-11-04 10:53:52,157] {hive_server_hook:162} INFO - Written 1030000 rows so far.
[2020-11-04 10:53:52,550] {hive_server_hook:162} INFO - Written 1040000 rows so far.
[2020-11-04 10:53:52,953] {hive_server_hook:162} INFO - Written 1050000 rows so far.
[2020-11-04 10:53:53,330] {hive_server_hook:162} INFO - Written 106000

[2020-11-04 10:54:27,890] {hive_server_hook:162} INFO - Written 1920000 rows so far.
[2020-11-04 10:54:28,295] {hive_server_hook:162} INFO - Written 1930000 rows so far.
[2020-11-04 10:54:28,704] {hive_server_hook:162} INFO - Written 1940000 rows so far.
[2020-11-04 10:54:29,116] {hive_server_hook:162} INFO - Written 1950000 rows so far.
[2020-11-04 10:54:29,531] {hive_server_hook:162} INFO - Written 1960000 rows so far.
[2020-11-04 10:54:29,923] {hive_server_hook:162} INFO - Written 1970000 rows so far.
[2020-11-04 10:54:30,348] {hive_server_hook:162} INFO - Written 1980000 rows so far.
[2020-11-04 10:54:30,765] {hive_server_hook:162} INFO - Written 1990000 rows so far.
[2020-11-04 10:54:31,188] {hive_server_hook:162} INFO - Written 2000000 rows so far.
[2020-11-04 10:54:31,563] {hive_server_hook:162} INFO - Written 2010000 rows so far.
[2020-11-04 10:54:31,964] {hive_server_hook:162} INFO - Written 2020000 rows so far.
[2020-11-04 10:54:32,369] {hive_server_hook:162} INFO - Written 2

[2020-11-04 10:55:09,712] {hive_server_hook:162} INFO - Written 490000 rows so far.
[2020-11-04 10:55:10,185] {hive_server_hook:162} INFO - Written 500000 rows so far.
[2020-11-04 10:55:10,645] {hive_server_hook:162} INFO - Written 510000 rows so far.
[2020-11-04 10:55:11,091] {hive_server_hook:162} INFO - Written 520000 rows so far.
[2020-11-04 10:55:11,554] {hive_server_hook:162} INFO - Written 530000 rows so far.
[2020-11-04 10:55:12,020] {hive_server_hook:162} INFO - Written 540000 rows so far.
[2020-11-04 10:55:12,535] {hive_server_hook:162} INFO - Written 550000 rows so far.
[2020-11-04 10:55:12,968] {hive_server_hook:162} INFO - Written 560000 rows so far.
[2020-11-04 10:55:13,439] {hive_server_hook:162} INFO - Written 570000 rows so far.
[2020-11-04 10:55:13,915] {hive_server_hook:162} INFO - Written 580000 rows so far.
[2020-11-04 10:55:14,373] {hive_server_hook:162} INFO - Written 590000 rows so far.
[2020-11-04 10:55:14,806] {hive_server_hook:162} INFO - Written 600000 rows 

/home/souche/projects/calculation/py3dev_new/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2,3,7,9,12,13,16,18,19,20,22,23,24,25,26,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
car_model = car_model[['model_code','series_code','brand_code']]
car_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65251 entries, 0 to 65250
Data columns (total 3 columns):
model_code     65251 non-null object
series_code    65251 non-null object
brand_code     65251 non-null object
dtypes: object(3)
memory usage: 1.5+ MB


In [6]:
def merge_data(df1, df2, anchor):
    cols_to_use = df2.columns.difference(df1.columns)
    df2_reindex = df2.set_index(anchor)[cols_to_use].reset_index()
    return df1.merge(df2_reindex, on = anchor, how = 'left')

dfc_sales_cleaned = merge_data(dfc_sales, car_model, 'model_code')
dfc_purchase_cleaned = merge_data(dfc_purchase, car_model, 'model_code')
dfc_sales_cleaned = dfc_sales_cleaned[~dfc_sales_cleaned['month'].isna()]
dfc_purchase_cleaned = dfc_purchase_cleaned[~dfc_purchase_cleaned['month'].isna()]
dfc_sales_cleaned = dfc_sales_cleaned[~dfc_sales_cleaned['canzhi'].isna()]
dfc_purchase_cleaned = dfc_purchase_cleaned[~dfc_purchase_cleaned['canzhi'].isna()]

dfc_sales_cleaned['month'] = dfc_sales_cleaned['month'].astype(int)
dfc_purchase_cleaned['month'] = dfc_purchase_cleaned['month'].astype(int)
dfc_sales_cleaned['canzhi'] = dfc_sales_cleaned['canzhi'].astype(float)
dfc_purchase_cleaned['canzhi'] = dfc_purchase_cleaned['canzhi'].astype(float)

median_sale = dfc_sales_cleaned.groupby(by = ['series_code', 'month'])['canzhi'].median().reset_index()
median_sale.columns = ['series_code', 'age', 'sale_residual_median']
median_purchase = dfc_purchase_cleaned.groupby(by = ['series_code', 'month'])['canzhi'].median().reset_index()
median_purchase.columns = ['series_code', 'age', 'purchase_residual_median']

profit_margin = pd.merge(median_sale, median_purchase, how = 'inner', on = ['series_code', 'age'])
profit_margin['margin'] = profit_margin['sale_residual_median'] - profit_margin['purchase_residual_median']

series_profit_margin_stats = profit_margin.groupby(by = 'series_code')['margin'].median().rename('series_margin_median').reset_index()
temp1 = profit_margin.groupby(by = 'series_code')['margin'].std().rename('series_margin_std').reset_index()
temp2 = profit_margin.groupby(by = 'series_code')['series_code'].count().rename('data_counts').reset_index()
series_profit_margin_stats = pd.merge(series_profit_margin_stats, temp1, how = 'left', on = 'series_code')
series_profit_margin_stats = pd.merge(series_profit_margin_stats, temp2, how = 'left', on = 'series_code')

# 车型库去重
car_model_no_duplicate = car_model[['series_code', 'brand_code']].drop_duplicates()

# series level merge
all_profit_margin_stats_series = series_profit_margin_stats.merge(car_model_no_duplicate, on = 'series_code' , how = 'right')[['series_code','series_margin_median','brand_code','data_counts']]
all_profit_margin_stats_series['data_counts'].fillna(0, inplace = True)

all_profit_margin_stats_brand = all_profit_margin_stats_series.query('data_counts >= 10 and series_margin_median >0').groupby(
    by='brand_code')['series_margin_median'].median().rename('brand_margin_median').reset_index()

all_profit_margin_stats = pd.merge(
    all_profit_margin_stats_series, all_profit_margin_stats_brand, how='left', on='brand_code')

# brand_margin_median填充负值、空值以及counts小于10
all_profit_margin_stats.loc[(all_profit_margin_stats['series_margin_median'] <= 0) | (all_profit_margin_stats['data_counts'] <= 10), 'series_margin_median'] = all_profit_margin_stats.loc[(
    all_profit_margin_stats['series_margin_median'] <= 0) | (all_profit_margin_stats['data_counts'] <= 10), 'brand_margin_median']

# 全局中位数继续填充空值
global_median = all_profit_margin_stats.loc[all_profit_margin_stats['series_margin_median'].notna(), 'series_margin_median'].median()
all_profit_margin_stats.loc[all_profit_margin_stats['series_margin_median'].isna(), 'series_margin_median'] = global_median

# margin小于1%的填充为车系25分位点
temp = all_profit_margin_stats.groupby(by='brand_code')['series_margin_median'].quantile(0.25).rename('series_margin_median_25').reset_index()
all_profit_margin_stats = pd.merge(all_profit_margin_stats, temp, how='left', on='brand_code')
all_profit_margin_stats.loc[all_profit_margin_stats['series_margin_median'] <= 0.01, 'series_margin_median'] = all_profit_margin_stats.loc[all_profit_margin_stats['series_margin_median'] <= 0.01, 'series_margin_median_25']

# margin仍然小于1%的用global_median填充
all_profit_margin_stats.loc[all_profit_margin_stats['series_margin_median'] <= 0.01, 'series_margin_median'] = global_median

result = pd.merge(car_model, all_profit_margin_stats, how = 'left', on = 'series_code')
result = result[['model_code', 'series_margin_median']]
result.columns = ['model_code', 'margin_median']



In [9]:
ph = curr_dir+"enterprise2_retial_purchase_margin.csv"
result.to_csv(ph)
from dayu.hooks.oss_hook import OSSHook
oss = OSSHook("oss_algorithm")
oss.put_file("algorithm/qiongjiu/valuation/2020-11-02/enterprise2_retial_purchase_margin.csv", ph)


[2020-11-04 10:55:32,590] {oss_hook:28} INFO - Done. Loaded the key algorithm/qiongjiu/valuation/2020-11-02/enterprise2_retial_purchase_margin.csv .
